In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
import uuid
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import textstat
import scml
import lalaes2 as mylib

In [2]:
version = "v01"
n_splits = 5
features = []
text_col = "clean_text"

In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [4]:
df = pd.read_parquet(f"input/val_{version}.parquet")
df = df.drop(columns=["source", "str_level"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   essay_id    866 non-null    object
 1   score       866 non-null    int8  
 2   clean_text  866 non-null    object
dtypes: int8(1), object(2)
memory usage: 14.5+ KB


# Character & Word-level Features

In [5]:
%%time
col = "cw_len"
df[col] = df[text_col].str.len()
features.append(col)

CPU times: user 1 ms, sys: 689 µs, total: 1.69 ms
Wall time: 1.58 ms


In [6]:
def digit_frac(row) -> float:
    return mylib.digit_frac(row[text_col])


def letter_frac(row) -> float:
    return mylib.letter_frac(row[text_col])


def space_frac(row) -> float:
    return mylib.space_frac(row[text_col])


def punc_frac(row) -> float:
    return mylib.punc_frac(row[text_col])


def upper_frac(row) -> float:
    return mylib.upper_frac(row[text_col])


def repeat_char_frac(row) -> float:
    return mylib.repeat_char_frac(row[text_col])


def repeat_substring_frac(row) -> float:
    return mylib.repeat_substring_frac(row[text_col])


def unique_word_frac(row) -> float:
    return mylib.unique_word_frac(row[text_col])


sf = mylib.StopwordFraction()


def stopword_frac(row) -> float:
    return sf(row[text_col])



cw_fns: List[Tuple[str, Callable]] = [
    ("cw_digit_frac", digit_frac),
    ("cw_letter_frac", letter_frac),
    ("cw_space_frac", space_frac),
    ("cw_punc_frac", punc_frac),
    ("cw_upper_frac", upper_frac),
    ("cw_repeat_char_frac", repeat_char_frac),
    ("cw_repeat_substring_frac", repeat_substring_frac),
    ("cw_unique_word_frac", unique_word_frac),
    ("cw_stopword_frac", stopword_frac),
]   
for col, fn in cw_fns:
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    features.append(col)
df[features] = df[features].astype(np.float32)

cw_digit_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 14761.41it/s]


cw_letter_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 14156.53it/s]


cw_space_frac


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:37<00:00, 23.14it/s]


cw_unique_word_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 30884.26it/s]


cw_stopword_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 28978.85it/s]


# Textstat Features

In [7]:
def syllable_count(row) -> int:
    return textstat.syllable_count(row[text_col])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row[text_col])


def sentence_count(row) -> int:
    return textstat.sentence_count(row[text_col])


def syllables_per_word(row) -> float:
    return row["ts_syllable_count"] / (row["ts_lexicon_count"] + 1)


def syllables_per_sent(row) -> float:
    return row["ts_syllable_count"] / (row["ts_sentence_count"] + 1)


def words_per_sent(row) -> float:
    return row["ts_lexicon_count"] / (row["ts_sentence_count"] + 1)


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row[text_col])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row[text_col])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row[text_col])


def smog_index(row) -> float:
    return textstat.smog_index(row[text_col])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row[text_col])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row[text_col])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row[text_col])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row[text_col])


def text_standard(row) -> float:
    return textstat.text_standard(row[text_col], float_output=True)


def mcalpine_eflaw(row) -> float:
    return textstat.mcalpine_eflaw(row[text_col])


textstat_fns: List[Tuple[str, Callable]] = [
    ("ts_syllable_count", syllable_count),
    ("ts_lexicon_count", lexicon_count),
    ("ts_sentence_count", sentence_count),
    ("ts_syllables_per_word", syllables_per_word),
    ("ts_syllables_per_sent", syllables_per_sent),
    ("ts_words_per_sent", words_per_sent),
    ("ts_flesch_reading_ease", flesch_reading_ease),
    ("ts_flesch_kincaid_grade", flesch_kincaid_grade),
    ("ts_gunning_fog", gunning_fog),
    ("ts_smog_index", smog_index),
    ("ts_automated_readability_index", automated_readability_index),
    ("ts_coleman_liau_index", coleman_liau_index),
    ("ts_linsear_write_formula", linsear_write_formula),
    ("ts_dale_chall_readability_score", dale_chall_readability_score),
    ("ts_text_standard", text_standard),
    ("ts_mcalpine_eflaw", mcalpine_eflaw),
]    
for col, fn in textstat_fns:
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    features.append(col)
df[features] = df[features].astype(np.float32)

ts_syllable_count


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 2444.05it/s]


ts_lexicon_count


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 28604.58it/s]


ts_sentence_count


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 2815.11it/s]


ts_text_standard


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:01<00:00, 818.89it/s]


ts_mcalpine_eflaw


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 866/866 [00:00<00:00, 9066.61it/s]


# Review Data

In [13]:
df[features] = df[features].astype(np.float32)
df = df.drop(columns=["clean_text"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866 entries, 0 to 865
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   essay_id                         866 non-null    object 
 1   score                            866 non-null    int8   
 2   cw_len                           866 non-null    float32
 3   cw_digit_frac                    866 non-null    float32
 4   cw_letter_frac                   866 non-null    float32
 5   cw_space_frac                    866 non-null    float32
 6   cw_punc_frac                     866 non-null    float32
 7   cw_upper_frac                    866 non-null    float32
 8   cw_repeat_char_frac              866 non-null    float32
 9   cw_repeat_substring_frac         866 non-null    float32
 10  cw_unique_word_frac              866 non-null    float32
 11  cw_stopword_frac                 866 non-null    float32
 12  ts_syllable_count     

In [14]:
df[features].describe(percentiles=percentiles)

,cw_digit_frac,cw_len,cw_letter_frac,cw_punc_frac,cw_repeat_char_frac,cw_repeat_substring_frac,cw_space_frac,cw_stopword_frac,cw_unique_word_frac,cw_upper_frac,ts_automated_readability_index,ts_coleman_liau_index,ts_dale_chall_readability_score,ts_flesch_kincaid_grade,ts_flesch_reading_ease,ts_gunning_fog,ts_lexicon_count,ts_linsear_write_formula,ts_mcalpine_eflaw,ts_sentence_count,ts_smog_index,ts_syllable_count,ts_syllables_per_sent,ts_syllables_per_word,ts_text_standard,ts_words_per_sent
count,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.000000,866.00000,866.000000
mean,0.002611,2060.635010,0.795851,0.020361,0.015119,0.002907,0.181177,0.504496,0.533583,0.018148,11.009238,8.253776,7.593972,9.188222,68.661377,10.897321,370.844116,10.499711,31.437298,19.625866,10.107390,509.431885,26.972654,1.362833,9.80485,19.788095
std,0.003620,849.165894,0.011515,0.006710,0.004385,0.007891,0.010704,0.048059,0.075234,0.009073,15.534221,1.855127,1.734166,12.169928,32.334080,12.454148,147.202759,5.296445,45.399860,8.809137,1.943423,212.012878,20.295984,0.092970,12.05920,15.702071
min,0.000000,732.000000,0.743163,0.000000,0.004212,0.000000,0.155241,0.354054,0.264228,0.000000,2.800000,3.190000,5.380000,2.200000,-628.880005,3.910000,151.000000,2.800000,12.100000,1.000000,0.000000,190.000000,10.962963,1.108434,0.00000,8.500000
1%,0.000000,843.000000,0.765810,0.006005,0.006511,0.000000,0.159538,0.385154,0.361339,0.000906,3.565000,3.959500,5.800000,3.265000,37.691500,5.416500,159.300000,4.235606,14.765000,4.000000,5.765000,205.600000,13.469216,1.153609,5.00000,10.195938
5%,0.000000,969.000000,0.776374,0.010542,0.008457,0.000000,0.164993,0.427421,0.410804,0.006928,5.400000,5.155000,6.172500,4.800000,53.077499,6.655000,180.000000,5.458333,18.425000,8.000000,7.400000,238.500000,16.406060,1.208756,6.00000,12.396826
10%,0.000000,1087.500000,0.781379,0.012364,0.009790,0.000000,0.168216,0.446998,0.439701,0.009094,6.150000,5.840000,6.400000,5.400000,57.400002,7.280000,202.000000,6.000000,19.900000,9.500000,7.900000,269.500000,17.884259,1.241224,6.00000,13.362500
20%,0.000000,1314.000000,0.786875,0.015158,0.011118,0.000000,0.171930,0.467442,0.466775,0.011138,7.300000,6.660000,6.710000,6.300000,61.900002,8.080000,241.000000,6.857143,21.700001,12.000000,8.700000,322.000000,19.769230,1.284289,7.00000,14.709678
30%,0.000000,1482.000000,0.790532,0.016810,0.012684,0.000778,0.174986,0.479837,0.493053,0.013238,8.100000,7.245000,6.970000,6.900000,65.730003,8.570000,274.000000,7.666667,23.400000,14.000000,9.300000,364.500000,21.358655,1.314428,7.00000,15.666667
40%,0.000835,1700.000000,0.793531,0.018519,0.013810,0.001460,0.177907,0.492891,0.514706,0.014957,8.900000,7.770000,7.230000,7.400000,68.599998,9.100000,310.000000,8.333333,24.799999,16.000000,9.700000,420.000000,22.853659,1.339181,8.00000,16.727272


In [15]:
features.sort()
with open(f"output/features_{version}.json", "w") as f:
    json.dump({"feature_names": features}, f)

In [16]:
df.to_parquet(f"output/features_{version}.parquet", index=False)
assert df.notna().all(axis=None)

In [17]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

RuntimeError: Not started